In [ ]:
import os
import requests
import json
from dotenv import load_dotenv

# .env 파일 로딩
load_dotenv()
CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")

# 공통 요청 함수
def send_naver_request(base_url, payload):
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET
    }

    try:
        res = requests.get(base_url, params=payload, headers=headers)
        res.raise_for_status()
        return res.json()
    except requests.exceptions.RequestException as e:
        print(f"요청 실패: {e}")
        return None

# 책 검색 + 저장 함수
def search_books(query):
    base_url = "https://openapi.naver.com/v1/search/book.json"
    payload = {
        "query": query,
        "display": 50,
        "sort": "sim"
    }

    result = send_naver_request(base_url, payload)

    if result and 'items' in result:
        items_data = result['items']

        # 저장할 디렉토리 없으면 생성
        os.makedirs("data", exist_ok=True)

        # JSON 파일로 저장
        with open("data/books.json", "w", encoding="utf-8") as f:
            json.dump(items_data, f, ensure_ascii=False, indent=4)

        print("data/books.json 파일로 저장 완료!")

        # Optional: 콘솔 출력
        for item in items_data:
            print(f"제목: {item['title']}")
            print(f"저자: {item['author']}")
            print(f"출판사: {item['publisher']}")
            print(f"링크: {item['link']}")
            print("-" * 50)
    else:
        print("검색 결과가 없습니다.")

# 실행 예시
if __name__ == "__main__":
    search_books("파이썬")


In [3]:
import pandas as pd

books_df = pd.read_json("data/books.json")

In [ ]:
books_df.head()

In [ ]:
books_df.loc[books_df['discount'] >= 20000, books_df.columns.drop(['link', 'image', 'isbn','description'])]\
    .sort_values(by='discount',ascending=False).reset_index(drop=True)

In [ ]:
books_df.loc[books_df['publisher'].str.contains('인피니티북스'), books_df.columns.drop(['image','description'])].reset_index(drop=True)

In [ ]:
import os
import requests
import json
from dotenv import load_dotenv

# .env 로드
load_dotenv()
CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")

# 공통 요청 함수
def send_naver_request(base_url, payload):
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET
    }

    try:
        res = requests.get(base_url, params=payload, headers=headers)
        res.raise_for_status()
        return res.json()
    except requests.exceptions.RequestException as e:
        print(f"요청 실패: {e}")
        return None

# 쇼핑 검색 함수
def search_shops(query):
    base_url = 'https://openapi.naver.com/v1/search/shop.json'
    payload = {
        "query": query,
        "display": 50,
        "sort": "sim"
    }

    result = send_naver_request(base_url, payload)

    if result and 'items' in result:
        # 필요한 필드만 추출
        extracted_items = []
        for item in result['items']:
            extracted = {
                "title": item.get("title"),
                "link": item.get("link"),
                "image": item.get("image"),
                "lprice": item.get("lprice"),
                "hprice": item.get("hprice", ""),  # 일부 상품엔 없음
                "mallName": item.get("mallName"),
                "productId": item.get("productId"),
                "productType": item.get("productType"),
                "brand": item.get("brand"),
                "maker": item.get("maker"),
                "category1": item.get("category1"),
                "category2": item.get("category2"),
                "category3": item.get("category3"),
                "category4": item.get("category4")
            }
            extracted_items.append(extracted)

        # 저장할 디렉토리 생성
        os.makedirs("data", exist_ok=True)

        # JSON 파일 저장
        with open("data/shops.json", "w", encoding="utf-8") as f:
            json.dump(extracted_items, f, ensure_ascii=False, indent=4)

        print("data/shops.json 파일로 저장\n")

    else:
        print("검색 결과가 없습니다.")

# 실행 예시
if __name__ == "__main__":
    search_shops("가디건")

In [ ]:
shops_df = pd.read_json("data/shops.json")
shops_df.head()

In [ ]:
shops_df.loc[shops_df['lprice'] <= 50000, shops_df.columns.drop(['title', 'image','hprice','productType','productId','maker','category1','category2','category3','category4'])] \
    .sort_values(by='lprice',ascending=True).reset_index(drop=True)

In [ ]:
shops_df.loc[shops_df['mallName'] == '네이버', 'lprice':'brand'] \
    .sort_values(by='lprice',ascending=True).reset_index(drop=True)